<a href="https://colab.research.google.com/github/kmenesesrivera/codonusagebias/blob/main/Codon_usage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
import tensorflow as tf
import sklearn as sk
import joblib
from google.colab import files
from sklearn.ensemble import RandomForestClassifier 
import pickle


data1 = pd.read_csv('https://drive.google.com/uc?export=download&id=1Z4v43cvTwp920NyOdboDKP7_ytC_0tBC')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (5,6) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
 
#Lectura del dataset
 
data1.shape

data1.head(5)
 

,Kingdom,DNAtype,SpeciesID,Ncodons,SpeciesName,UUU,UUC,UUA,UUG,CUU,CUC,CUA,CUG,AUU,AUC,AUA,AUG,GUU,GUC,GUA,GUG,GCU,GCC,GCA,GCG,CCU,CCC,CCA,CCG,UGG,GGU,GGC,GGA,GGG,UCU,UCC,UCA,UCG,AGU,AGC,ACU,ACC,ACA,ACG,UAU,UAC,CAA,CAG,AAU,AAC,UGU,UGC,CAU,CAC,AAA,AAG,CGU,CGC,CGA,CGG,AGA,AGG,GAU,GAC,GAA,GAG,UAA,UAG,UGA
0,vrl,0,100217,1995,Epizootic haematopoietic necrosis virus,0.01654,0.01203,0.00050,0.00351,0.01203,0.03208,0.00100,0.04010,0.00551,0.02005,0.00752,0.02506,0.01103,0.04110,0.00902,0.03308,0.01003,0.05013,0.01554,0.01103,0.02356,0.03208,0.01203,0.00501,0.01003,0.01203,0.03158,0.01905,0.02456,0.01353,0.02155,0.00251,0.00652,0.00150,0.01554,0.00501,0.02105,0.00902,0.01053,0.00501,0.02256,0.00301,0.03108,0.00401,0.02607,0.00251,0.01153,0.00501,0.02356,0.01053,0.03860,0.00401,0.00702,0.00401,0.00451,0.01303,0.03559,0.01003,0.04612,0.01203,0.04361,0.00251,0.00050,0.00000
1,vrl,0,100220,1474,Bohle iridovirus,0.02714,0.01357,0.00068,0.00678,0.00407,0.02849,0.00204,0.04410,0.01153,0.02510,0.00882,0.03324,0.00814,0.04071,0.00814,0.03256,0.01085,0.04885,0.01221,0.01357,0.00678,0.02714,0.01221,0.00407,0.01425,0.01221,0.01967,0.02239,0.01289,0.02103,0.01493,0.00407,0.00475,0.00068,0.02035,0.00950,0.02782,0.01425,0.00611,0.00475,0.02917,0.00407,0.02374,0.00882,0.02917,0.00271,0.01628,0.00204,0.01967,0.00543,0.03392,0.00136,0.00678,0.00136,0.00136,0.01696,0.03596,0.01221,0.04545,0.01560,0.04410,0.00271,0.00068,0.00000
2,vrl,0,100755,4862,Sweet potato leaf curl virus,0.01974,0.0218,0.01357,0.01543,0.00782,0.01111,0.01028,0.01193,0.02283,0.01604,0.01316,0.02180,0.01625,0.01872,0.01213,0.01070,0.02406,0.01234,0.01440,0.00514,0.01604,0.01460,0.02098,0.01070,0.01728,0.01851,0.00864,0.01172,0.01892,0.01933,0.01419,0.01296,0.00967,0.01337,0.01337,0.01851,0.01131,0.01419,0.01090,0.02612,0.01275,0.01522,0.02365,0.02962,0.01789,0.01625,0.01234,0.01604,0.01687,0.02077,0.03949,0.00864,0.00596,0.00926,0.00596,0.01974,0.02489,0.03126,0.02036,0.02242,0.02468,0.00391,0.00000,0.00144
3,vrl,0,100880,1915,Northern cereal mosaic virus,0.01775,0.02245,0.01619,0.00992,0.01567,0.01358,0.00940,0.01723,0.02402,0.02245,0.02507,0.02924,0.02089,0.02141,0.01723,0.01932,0.02141,0.00679,0.02245,0.00522,0.01358,0.00418,0.01410,0.00574,0.01201,0.00992,0.00366,0.02402,0.02663,0.02872,0.00992,0.02350,0.00522,0.01619,0.00836,0.02037,0.01358,0.02089,0.00731,0.02141,0.00888,0.01567,0.01253,0.02298,0.01358,0.00992,0.00888,0.00783,0.00679,0.03133,0.04282,0.00627,0.00261,0.00261,0.00366,0.01410,0.01671,0.03760,0.01932,0.03029,0.03446,0.00261,0.00157,0.00000
4,vrl,0,100887,22831,Soil-borne cereal mosaic virus,0.02816,0.01371,0.00767,0.03679,0.01380,0.00548,0.00473,0.02076,0.02716,0.00867,0.01310,0.02773,0.02803,0.00508,0.00920,0.02965,0.02878,0.00574,0.01572,0.01577,0.01007,0.00508,0.00604,0.00679,0.01205,0.03127,0.00775,0.00959,0.00797,0.02006,0.00359,0.00933,0.01191,0.01616,0.00788,0.02593,0.00854,0.01200,0.02098,0.02089,0.01367,0.01502,0.01809,0.02738,0.01796,0.01082,0.00705,0.01174,0.00858,0.03408,0.03964,0.00950,0.00429,0.00578,0.00604,0.01494,0.01734,0.04148,0.02483,0.03359,0.03679,0.00000,0.00044,0.00131


In [ ]:
def preprocess_dataset(dataset, save_metadata=True):
  
  preprocessed_dataset = dataset.copy()

  #Determinamos aquellas variables que tengan un porcentaje de registros únicos por cada variable mayor al valor de 0.7
  threshold = 0.7

  unique_percentages = dataset.nunique() / len(dataset)

  criteria = unique_percentages > threshold

  columns_to_filter = unique_percentages[criteria].keys()

  #Eliminamos las variables seleccionadas en el paso anterior
  preprocessed_dataset.drop(columns_to_filter, axis=1, inplace=True)

  #Eliminamos los registros duplicados
  preprocessed_dataset.drop_duplicates(keep="first", inplace=True)

  #Determinamos que variables son del tipo númerico y cuales son categóricas
  numeric_columns = list()
  categorical_columns = list()
  dictionary_of_columns_with_index_to_categorical = dict()
  dictionary_of_columns_with_categorical_to_index = dict()

  for column in preprocessed_dataset:
    #Determinamos si la variable es numérica o no
    if pd.api.types.is_numeric_dtype(preprocessed_dataset[column]):
      numeric_columns.append(column)
    else:
      #Modificamos el tipo de dato de la variable mediante "astype"
      preprocessed_dataset[column] = preprocessed_dataset[column].astype("category")

      #Verificamos si el tipo de dato de la variable fue transformado a categórico correctamente
      if not pd.api.types.is_categorical_dtype(preprocessed_dataset[column]):
        raise Exception("La columna {} no se transformó correctamente a categórica".format(column))

      dictionary_of_columns_with_index_to_categorical[column] = dict()
      dictionary_of_columns_with_categorical_to_index[column] = dict()
      
      #Indexamos los valores (categorías), sin tomar en consideración los nulos, de la variable y guardamos esa información en los diccionarios
      for index, category in enumerate(preprocessed_dataset[column].cat.categories):
        dictionary_of_columns_with_index_to_categorical[column][index] = category
        dictionary_of_columns_with_categorical_to_index[column][category] = index
      
      categorical_columns.append(column)
  
  #Reemplazamos los nulos con la mediana sólo de aquellas variables numéricas
  #  median_of_numeric_columns = preprocessed_dataset[numeric_columns].median()

  #  preprocessed_dataset[numeric_columns] = preprocessed_dataset[numeric_columns].fillna(median_of_numeric_columns)

  #Transformamos a números los valores (categorías) de las variables categóricas sin considerar los nulos
  preprocessed_dataset.replace(dictionary_of_columns_with_categorical_to_index, inplace=True)

  #Obtenemos si los registros cumplen o no con la condición de tener nulos en la variable "Embarked"
  #criteria = preprocessed_dataset.Embarked.isna()

  #Eliminamos los nulos de la variable "Embarked"
  #preprocessed_dataset.drop(preprocessed_dataset[criteria].index, inplace=True)

  #Determinamos aquellas variables que tengan un porcentaje de registros nulos por cada variable mayor al valor de 0.7
  threshold = 0.7

  null_percentages = preprocessed_dataset[categorical_columns].isna().sum() / len(preprocessed_dataset)

  criteria = null_percentages > threshold

  columns_to_filter = null_percentages[criteria].keys()

  #Eliminamos las variables seleccionadas en el paso anterior
  preprocessed_dataset.drop(columns_to_filter, axis=1, inplace=True)

  #Eliminamos los registros duplicados
  preprocessed_dataset.drop_duplicates(keep="first", inplace=True)

  if save_metadata:
    #Guardamos información que consideremos imporante. **El formato pickle nos permite guardar objetos de python***
    with open("numeric_columns.pkl", "wb") as f:
      pickle.dump(numeric_columns, f)
    
    with open("categorical_columns.pkl", "wb") as f:
      pickle.dump(categorical_columns, f)
    
    with open("dictionary_of_columns_with_index_to_categorical.pkl", "wb") as f:
      pickle.dump(dictionary_of_columns_with_index_to_categorical, f)
    
    with open("dictionary_of_columns_with_categorical_to_index.pkl", "wb") as f:
      pickle.dump(dictionary_of_columns_with_categorical_to_index, f)
  
  return preprocessed_dataset

In [ ]:
#Transformamos los datos a valores numéricos
preprocessed_dataset = preprocess_dataset(data1)

In [ ]:
preprocessed_dataset.head(10)

,Kingdom,DNAtype,Ncodons,UUU,UUC,UUA,UUG,CUU,CUC,CUA,CUG,AUU,AUC,AUA,AUG,GUU,GUC,GUA,GUG,GCU,GCC,GCA,GCG,CCU,CCC,CCA,CCG,UGG,GGU,GGC,GGA,GGG,UCU,UCC,UCA,UCG,AGU,AGC,ACU,ACC,ACA,ACG,UAU,UAC,CAA,CAG,AAU,AAC,UGU,UGC,CAU,CAC,AAA,AAG,CGU,CGC,CGA,CGG,AGA,AGG,GAU,GAC,GAA,GAG,UAA,UAG,UGA
0,9,0,1995,4188,3559,0.00050,0.00351,0.01203,0.03208,0.00100,0.04010,0.00551,0.02005,0.00752,0.02506,0.01103,0.04110,0.00902,0.03308,0.01003,0.05013,0.01554,0.01103,0.02356,0.03208,0.01203,0.00501,0.01003,0.01203,0.03158,0.01905,0.02456,0.01353,0.02155,0.00251,0.00652,0.00150,0.01554,0.00501,0.02105,0.00902,0.01053,0.00501,0.02256,0.00301,0.03108,0.00401,0.02607,0.00251,0.01153,0.00501,0.02356,0.01053,0.03860,0.00401,0.00702,0.00401,0.00451,0.01303,0.03559,0.01003,0.04612,0.01203,0.04361,0.00251,0.00050,0.00000
1,9,0,1474,5156,3703,0.00068,0.00678,0.00407,0.02849,0.00204,0.04410,0.01153,0.02510,0.00882,0.03324,0.00814,0.04071,0.00814,0.03256,0.01085,0.04885,0.01221,0.01357,0.00678,0.02714,0.01221,0.00407,0.01425,0.01221,0.01967,0.02239,0.01289,0.02103,0.01493,0.00407,0.00475,0.00068,0.02035,0.00950,0.02782,0.01425,0.00611,0.00475,0.02917,0.00407,0.02374,0.00882,0.02917,0.00271,0.01628,0.00204,0.01967,0.00543,0.03392,0.00136,0.00678,0.00136,0.00136,0.01696,0.03596,0.01221,0.04545,0.01560,0.04410,0.00271,0.00068,0.00000
2,9,0,4862,4476,4492,0.01357,0.01543,0.00782,0.01111,0.01028,0.01193,0.02283,0.01604,0.01316,0.02180,0.01625,0.01872,0.01213,0.01070,0.02406,0.01234,0.01440,0.00514,0.01604,0.01460,0.02098,0.01070,0.01728,0.01851,0.00864,0.01172,0.01892,0.01933,0.01419,0.01296,0.00967,0.01337,0.01337,0.01851,0.01131,0.01419,0.01090,0.02612,0.01275,0.01522,0.02365,0.02962,0.01789,0.01625,0.01234,0.01604,0.01687,0.02077,0.03949,0.00864,0.00596,0.00926,0.00596,0.01974,0.02489,0.03126,0.02036,0.02242,0.02468,0.00391,0.00000,0.00144
3,9,0,1915,4295,4555,0.01619,0.00992,0.01567,0.01358,0.00940,0.01723,0.02402,0.02245,0.02507,0.02924,0.02089,0.02141,0.01723,0.01932,0.02141,0.00679,0.02245,0.00522,0.01358,0.00418,0.01410,0.00574,0.01201,0.00992,0.00366,0.02402,0.02663,0.02872,0.00992,0.02350,0.00522,0.01619,0.00836,0.02037,0.01358,0.02089,0.00731,0.02141,0.00888,0.01567,0.01253,0.02298,0.01358,0.00992,0.00888,0.00783,0.00679,0.03133,0.04282,0.00627,0.00261,0.00261,0.00366,0.01410,0.01671,0.03760,0.01932,0.03029,0.03446,0.00261,0.00157,0.00000
4,9,0,22831,5246,3717,0.00767,0.03679,0.01380,0.00548,0.00473,0.02076,0.02716,0.00867,0.01310,0.02773,0.02803,0.00508,0.00920,0.02965,0.02878,0.00574,0.01572,0.01577,0.01007,0.00508,0.00604,0.00679,0.01205,0.03127,0.00775,0.00959,0.00797,0.02006,0.00359,0.00933,0.01191,0.01616,0.00788,0.02593,0.00854,0.01200,0.02098,0.02089,0.01367,0.01502,0.01809,0.02738,0.01796,0.01082,0.00705,0.01174,0.00858,0.03408,0.03964,0.00950,0.00429,0.00578,0.00604,0.01494,0.01734,0.04148,0.02483,0.03359,0.03679,0.00000,0.00044,0.00131
5,9,0,5274,5037,4529,0.01479,0.01024,0.02294,0.00758,0.01782,0.01403,0.02636,0.01327,0.01896,0.02579,0.01877,0.01346,0.00721,0.01782,0.02067,0.02313,0.01214,0.00265,0.01232,0.01953,0.02105,0.00322,0.01232,0.01631,0.01327,0.02560,0.01119,0.02010,0.01763,0.00986,0.00360,0.00929,0.01138,0.03527,0.03015,0.02844,0.00284,0.01555,0.02825,0.01953,0.01251,0.03906,0.03546,0.01138,0.00948,0.00683,0.01043,0.03223,0.00986,0.00398,0.00853,0.00322,0.00303,0.01593,0.00171,0.02427,0.02503,0.02825,0.01270,0.00133,0.00038,0.00209
6,9,0,3042,6342,3881,0.02005,0.02400,0.02761,0.01611,0.01052,0.00493,0.02597,0.00888,0.01512,0.02268,0.02893,0.00789,0.01151,0.01611,0.04011,0.01151,0.01118,0.00427,0.02794,0.01085,0.01216,0.00131,0.01216,0.02137,0.01052,0.01512,0.00986,0.03846,0.01085,0.01249,0.00066,0.01085,0.00427,0.02728,0.01381,0.01315,0.00427,0.02465,0.01151,0.02465,0.01085,0.03452,0.01348,0.01545,0.00690,0.01512,0.00756,0.02926,0.02696,0.00888,0.00625,0.00690,0.00329,0.01315,0.00822,0.04011,0.01183,0.02663,0.02663,0.00033,0.00033,0.00000
7,9,0,2801,4770,4963,0.01214,0.02321,0.01714,0.02213,0.00893,0.01928,0.01785,0.02356,0.01107,0.0199

In [ ]:
preprocessed_dataset.describe

<bound method NDFrame.describe of        Kingdom  DNAtype   Ncodons   UUU  ...      GAG      UAA      UAG      UGA
0            9        0      1995  4188  ...  0.04361  0.00251  0.00050  0.00000
1            9        0      1474  5156  ...  0.04410  0.00271  0.00068  0.00000
2            9        0      4862  4476  ...  0.02468  0.00391  0.00000  0.00144
3            9        0      1915  4295  ...  0.03446  0.00261  0.00157  0.00000
4            9        0     22831  5246  ...  0.03679  0.00000  0.00044  0.00131
...        ...      ...       ...   ...  ...      ...      ...      ...      ...
13023        7        0      1097  1508  ...  0.04102  0.00091  0.00091  0.00638
13024        7        1      2067   551  ...  0.00677  0.00242  0.00097  0.01887
13025        7        1      1686   672  ...  0.00297  0.00356  0.00119  0.02017
13026        7        0  40662582   946  ...  0.03959  0.00099  0.00079  0.00156
13027        7        1   8998998   963  ...  0.00783  0.00156  0.00114  0.

In [ ]:
x = preprocessed_dataset.drop("Kingdom", axis=1)
x = preprocessed_dataset.drop("DNAtype", axis=1)
y = preprocessed_dataset["Kingdom"]  
y2 = preprocessed_dataset["DNAtype"]  

#Mediante el método "train_test_split" usaremos el 20% de la data para probar el modelo. El parámetro "random state" nos sirve para
#poder replicar la misma separación
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=70)

 

In [ ]:
print(f"Train shape: {x_train.shape}")
print(f"Test shape: {x_test.shape}")

Train shape: (10393, 66)
Test shape: (2599, 66)


In [ ]:
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV

param_grid = {
 'max_depth':range(5,6)
}

xgb_Model = XGBClassifier(learning_rate=0.01, n_estimators=80, min_child_weight=2, gamma=0, colsample_bytree=0.8, subsample= 0.7, objective= 'binary:logistic', nthread=4, scale_pos_weight=1, seed=27, n_jobs=-1,scoring='roc_auc')
xgb_Grid = GridSearchCV (estimator= xgb_Model, param_grid = param_grid )
 
xgb_Grid.fit(x_train,y_train)
 
print ("optimal max_depth",xgb_Grid.best_estimator_.max_depth) 
print ("optimal xgb_Grid.best_score_",xgb_Grid.best_score_)
 

#clf = XGBClassifier(n_estimators=80, learning_rate=0.01,  max_depth=10,
#                    min_child_weight=2, 
#                    gamma=0,
#                    subsample=0.7,
#                    colsample_bytree=0.8,
#                    objective='binary:logistic',
#                    nthread=4,
#                    scale_pos_weight=1,
#                    seed=27,
#                    n_jobs=-1 ).fit(x_train, y_train)


optimal max_depth 5
optimal xgb_Grid.best_score_ 0.999422706833679


In [ ]:
xgb_Grid.score(x_train, y_train)

0.9998075627826422

In [ ]:
xgb_Grid.score(x_test, y_test)

0.9984609465178915

In [ ]:

#Mediante el método "train_test_split" usaremos el 20% de la data para probar el modelo. El parámetro "random state" nos sirve para
#poder replicar la misma separación
x_train, x_test, y2_train, y2_test = train_test_split(x, y2, test_size=0.2, random_state=70)


In [ ]:
print(f"Train shape: {x_train.shape}")
print(f"Test shape: {x_test.shape}")

Train shape: (10393, 66)
Test shape: (2599, 66)


In [ ]:
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV

param_grid = {
 'max_depth':range(9,10)
}

xgb_Model2 = XGBClassifier(learning_rate=0.01, n_estimators=80, min_child_weight=2, gamma=0, colsample_bytree=0.8, subsample= 0.7, objective= 'binary:logistic', nthread=4, scale_pos_weight=1, seed=27, n_jobs=-1,scoring='roc_auc')
xgb_Grid2 = GridSearchCV (estimator= xgb_Model2, param_grid = param_grid )
 
xgb_Grid2.fit(x_train,y2_train)
 
print ("optimal max_depth",xgb_Grid2.best_estimator_.max_depth)
print ("optimal xgb_Grid2.best_score_",xgb_Grid2.best_score_)
 

#clf = XGBClassifier(n_estimators=80, learning_rate=0.01,  max_depth=10,
#                    min_child_weight=2, 
#                    gamma=0,
#                    subsample=0.7,
#                    colsample_bytree=0.8,
#                    objective='binary:logistic',
#                    nthread=4,
#                    scale_pos_weight=1,
#                    seed=27,
#                    n_jobs=-1 ).fit(x_train, y2_train)
xgb_Grid2.score(x_test, y2_test)

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)


optimal max_depth 9
optimal xgb_Grid2.best_score_ 0.9853751780604524


0.9903809157368219

In [ ]:
xgb_Grid2.score(x_train, y2_train)

0.9924949485230443

In [ ]:
import seaborn as sns
from sklearn.metrics import roc_auc_score
X=[]
Y=[]
train_auc= [] 
rf_Model= XGBClassifier(learning_rate=0.01,max_depth=9 ,n_estimators=80, min_child_weight=2, gamma=0, colsample_bytree=0.8, subsample= 0.7, objective= 'binary:logistic', nthread=4, scale_pos_weight=1, seed=27, n_jobs=-1,scoring='roc_auc')
rf_Model.fit(x_train,y_train)
pred_train= rf_Model.predict(x_train)
X.append(80)
Y.append(9)
train_auc.append(roc_auc_score(y_train,pred_train)) 

ValueError: ignored

In [ ]:

y_pred_proba = rfcl.predict_proba(x_test)[::,1]
fpr, tpr, th = roc_curve(y_test,  y_pred_proba)
auc = roc_auc_score(y_test, y_pred_proba)
plt.title("Curva ROC y AUC")
plt.plot(fpr,tpr,label="data 1, auc="+str(auc))
plt.xlabel("FPR")
plt.ylabel("TPR")
plt.legend(loc=4)
plt.show()